# Action detection

In [3]:
experiment_name = 'action_classification'

import os
from detectron2.utils.logger import setup_logger
setup_logger()

from helpers.init import initialization
cfg, classes, labels_dict, train_dataset_combined, test_dataset_combined, valid_dataset_combined = initialization(merge_to_one_class = False)

### Training of the neural network ###

#cfg.MODEL.WEIGHTS = "include/maskdino_swinl_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask52.3ap_box59.0ap.pth" 
cfg.MODEL.WEIGHTS = "include/1_class_0.97.pth"

cfg.SOLVER.BACKBONE_MULTIPLIER = 1e-5

cfg.SOLVER.IMS_PER_BATCH = 2 # batch size
cfg.DATALOADER.NUM_WORKERS = 2
cfg.DATALOADER.PREFETCH_FACTOR = 2


cfg.SOLVER.BASE_LR = 1e-4
cfg.SOLVER.MAX_ITER = 5000 
cfg.SOLVER.STEPS = ([4000]) # when to decay learning rate

cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.TEST.EVAL_PERIOD = 500

dir_list = os.listdir('output')
if '.ipynb_checkpoints' in dir_list:
        dir_list.remove('.ipynb_checkpoints')

cfg.OUTPUT_DIR = os.path.join('output', f'{len(dir_list)}')

 100% |█████████████████| 233/233 [729.6ms elapsed, 0s remaining, 319.3 samples/s]      
 100% |███████████████████| 51/51 [155.0ms elapsed, 0s remaining, 333.1 samples/s] 
 100% |███████████████████| 50/50 [165.2ms elapsed, 0s remaining, 305.6 samples/s] 
Found train dataset labels:
['sitting', 'walking', 'waving']
Found test dataset labels:
['sitting', 'walking', 'waving']
Found valid dataset labels:
['sitting', 'walking', 'waving']
 100% |███████████████████| 74/74 [203.0ms elapsed, 0s remaining, 364.5 samples/s]     
 100% |███████████████████| 17/17 [66.4ms elapsed, 0s remaining, 256.0 samples/s]     
 100% |███████████████████| 17/17 [63.4ms elapsed, 0s remaining, 268.1 samples/s]     
Found difficult_train dataset labels:
['sitting', 'walking', 'waving']
Found difficult_test dataset labels:
['sitting', 'walking', 'waving']
Found difficult_valid dataset labels:
['sitting', 'walking', 'waving']
Means of the dataset:  [125.02593479  58.33375975 115.78446252 234.51154039]
STDs of th

## Training model

In [2]:
import logging
import json

# setup and launch the trainer
from helpers.model_trainer import Trainer
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

os.makedirs(cfg.OUTPUT_DIR)

# dump labels for deployment
import json
with open(os.path.join(cfg.OUTPUT_DIR, "classes.json"), "w") as fp:
    json.dump(classes, fp)

TRAIN = True
if TRAIN:
    trainer = Trainer(cfg, freeze=False, MaskDINO_weights=False)
    trainer.train()

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

KeyboardInterrupt: 

## Loading model

In [4]:
from helpers.benchmark import benchmark

# if we just want to load the results of a previous model use that models experiment_name here
# else leave model_name at None to use the most recently trained model

model_name = None
#model_name = 'model_0000499.pth'

weights_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") if model_name is None else os.path.join(cfg.OUTPUT_DIR, model_name, 'model.pth')
best_weights_path = os.path.join('include', '3_classes_0.92_unfrozen.pth')

cfg.MODEL.WEIGHTS =  weights_path
#cfg.MODEL.WEIGHTS =  best_weighst_path

# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
# cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.8

benchmark(cfg, classes, labels_dict, train_dataset_combined, test_dataset_combined, valid_dataset_combined)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Evaluating detections...
 100% |███████████████████| 68/68 [587.0ms elapsed, 0s remaining, 115.9 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 68/68 [588.9ms elapsed, 0s remaining, 115.5 samples/s]      
              precision    recall  f1-score   support

     sitting       1.00      0.95      0.97        40
     walking       0.85      0.85      0.85        27
      waving       0.86      0.96      0.91        26

   micro avg       0.91      0.92      0.92        93
   macro avg       0.90      0.92      0.91        93
weighted avg       0.92      0.92      0.92        93

VALIDATION DATASET

Evaluating detections...
 100% |███████████████████| 67/67 [606.0ms elapsed, 0s remaining, 110.6 samples/s]      
Performing IoU sweep...
 100% |███████████████████| 67/67 [661.4ms elapsed, 0s remaining, 101.3 samples/s]      
              precision    recall  f1-score   support

     sitting       0.98      1.00      0.99        43
     walking       0.81      0.79   

In [5]:
# launch a fiftyone session to view the model predictions after training
import fiftyone as fo
session = fo.launch_app(test_dataset_combined, port = 6006)
session.wait()

Notebook sessions cannot wait


In [6]:
fo.close_app()